# Labo 6 : Extraction d’attributs à partir de textes en vue de la classification$
**Autheurs :** Julien Mulhemman et Cristhian Ronquillo

### 1. Importer le corpus Reuters-21578 dans un notebook Jupyter grâce à NLTK. Bien identifier les documents d’entraînement (train) et ceux d’évaluation (test). N’utiliser que les premiers pour la recherche des paramètres optimaux.

In [66]:
import nltk
#nltk.download('reuters')

In [67]:
from nltk.corpus import reuters
print(reuters.readme())


      The Reuters-21578 benchmark corpus, ApteMod version

This is a publically available version of the well-known Reuters-21578
"ApteMod" corpus for text categorization.  It has been used in
publications like these:

 * Yiming Yang and X. Liu. "A re-examination of text categorization
   methods".  1999.  Proceedings of 22nd Annual International SIGIR.
   http://citeseer.nj.nec.com/yang99reexamination.html

 * Thorsten Joachims. "Text categorization with support vector
   machines: learning with many relevant features".  1998. Proceedings
   of ECML-98, 10th European Conference on Machine Learning.
   http://citeseer.nj.nec.com/joachims98text.html

ApteMod is a collection of 10,788 documents from the Reuters financial
newswire service, partitioned into a training set with 7769 documents
and a test set with 3019 documents.  The total size of the corpus is
about 43 MB.  It is also available for download from
http://kdd.ics.uci.edu/databases/reuters21578/reuters21578.html ,
which includ

In [68]:
print(reuters.fileids()[150:155])
print('training files : ', len([fid for fid in reuters.fileids() if fid[:5] == 'train']))
print('testing files : ', len([fid for fid in reuters.fileids() if fid[:4] == 'test']))
print('total files : ', len(reuters.fileids()))
print(reuters.words('test/15120')[:200])
print(reuters.categories('test/15120'))

['test/15119', 'test/15120', 'test/15121', 'test/15122', 'test/15124']
training files :  7769
testing files :  3019
total files :  10788
['CELLTRONICS', '&', 'lt', ';', 'CELT', '>', 'DECLARES', 'ONE', '-', 'FOR', '-', '10', 'SPLIT', 'Celltronics', 'Inc', 'said', 'its', 'directors', 'declared', 'a', 'one', '-', 'for', '-', '10', 'reverse', 'stock', 'split', 'of', 'its', 'common', 'stock', '.', 'It', 'said', 'the', 'split', 'will', 'be', 'payable', 'April', '17', 'to', 'shareholders', 'of', 'record', 'April', '16', '.']
['earn']


In [69]:
train_data = [fid for fid in reuters.fileids() if fid[:5] == 'train']
test_data = [fid for fid in reuters.fileids() if fid[:4] == 'test']

In [70]:
# Notez que le paramètre `test_size` est défini à 0.25 pour utiliser les 25% des fichiers d'entraînement comme évaluation.
from sklearn.model_selection import train_test_split

train_data, test_data = train_test_split(train_data, test_size=0.25, random_state=42)

print("Train data:", len(train_data))
print("Test data:", len(test_data))

Train data: 5826
Test data: 1943


Exemple d'accès au contenu d'un document

In [71]:
# Affiche le contenu brut du premier document d'entraînement
print(reuters.raw(train_data[0]))

CHRYSLER SAYS AMC HOLDERS TO GET CHRYSLER STOCK WORTH FOUR DLRS A SHARE UNDER BUYOUT PLAN

  CHRYSLER SAYS AMC HOLDERS TO GET CHRYSLER STOCK WORTH FOUR DLRS A SHARE UNDER BUYOUT PLAN
  




In [72]:
print(reuters.raw(test_data[0]))   # Affiche le contenu brut du premier document d'évaluation

ITEL &lt;ITEL> BUYS CASTLE/COOKE &lt;CKE> UNIT ASSETS
  Itel Corp said it has completed the
  previously-announced acquisition of the container fleet and
  related assets of Castle and Cooke Inc's Flexi-Van Leasing Inc
  subsidiary for about 130 mln dlrs in cash and marketable
  securities, 30 mln dlrs in notes, three mln common shares and
  the assumption of liabilities.
      The company said it has obtained 150 mln dlrs in financing
  from a bank group for the deal, and the common shares are
  subject to a 10-year restriction on their sale and voting.
  




### 2. Transformer les données (train et test) en DataFrames de pandas avec deux colonnes : le texte complet de la dépêche (réassemblé à partir des tokens de NLTK avec ‘ ‘.join()) et la catégorie ‘grain’ ou non-‘grain’, codées respectivement comme 1 et 0 (pour pouvoir appliquer facilement ‘f1-score’ en mode par défaut binaire avec ‘1’ comme classe positive).

In [73]:
import pandas as pd
from nltk.tokenize import word_tokenize

In [75]:
# Fonction pour extraire le texte et les catégories des documents
def get_text_and_categories(doc_ids):
    texts = []
    categories = []
    for doc_id in doc_ids:
        # Réassembler le texte complet à partir des tokens
        text = ' '.join(reuters.words(doc_id))
        # Encoder la catégorie 'grain' comme 1 et les autres comme 0
        category = 1 if 'grain' in reuters.categories(doc_id) else 0
        texts.append(text)
        categories.append(category)
    return texts, categories

In [76]:
# Obtenir les textes et les catégories des documents d'entraînement
train_texts, train_categories = get_text_and_categories(train_data)

# Obtenir les textes et les catégories des documents de test
test_texts, test_categories = get_text_and_categories(test_data)

In [77]:
# Créer des DataFrames pour les données d'entraînement et de test
train_df = pd.DataFrame({'texte': train_texts, 'categorie': train_categories})
test_df = pd.DataFrame({'texte': test_texts, 'categorie': test_categories})

In [78]:
# Afficher les premières lignes des DataFrames pour vérifier
print("DataFrame d'entraînement:")
print(train_df.head())

print("\nDataFrame de test:")
print(test_df.head())

DataFrame d'entraînement:
                                               texte  categorie
0  CHRYSLER SAYS AMC HOLDERS TO GET CHRYSLER STOC...          0
1  SCHWAB SAFE CO & lt ; SS > UPS PAYOUT Qtly div...          0
2  HAITI ALLOWS IMPORTATION OF BANNED BLACK PIGS ...          0
3  BEGHIN - SAY SEES SOLUTION TO EC SUGAR DISPUTE...          0
4  Japan February trade surplus 8 . 14 billion dl...          0

DataFrame de test:
                                               texte  categorie
0  ITEL & lt ; ITEL > BUYS CASTLE / COOKE & lt ; ...          0
1  MAGMA LOWERS COPPER 0 . 75 CENT TO 66 CTS Magm...          0
2  ECUADOR QUAKE TO CUT OIL REVENUES 800 MLN DLRS...          0
3  PUBLIC SERVICE N . C . & lt ; PSNC > RAISES PA...          0
4  AMERICAN STORES CO 4TH QTR SHR 1 . 57 DLRS VS ...          0


Maintenant, nous avons deux DataFrames `train_df` et `test_df` avec les colonnes 'text' et 'label'. La colonne
'text' contient le texte complet de la dépêche, et la colonne 'label' contient la catégorie 'grain' ou
non-'grain', codée respectivement comme 1 et 0.

### 3. Adapter le code fourni par Scikit-learn pour la classification de 20 Newsgroups (un notebook est également disponible) afin de trouver les meilleurs hyperparamètres pour la classification des dépêches de Reuters-21578 en ‘grain’ et non-‘grain’, sans utiliser les données de test. 

>Procéder par validation croisée (GridSearchCV) sur les données d’entraînement et à la fin indiquer clairement dans votre notebook les hyperparamètres optimaux trouvés (faites une conclusion sous forme de tableau). Utiliser comme métrique le score f1 de la classe ‘grain’.  

>Veuillez explorer d’autres options pour CountVectorizer et TfidfTransformer que celles fournies dans l’exemple de Scikit-learn.

>Comparer les résultats obtenus avec deux classifieurs : le modèle bayésien ComplementNB fourni dans l’exemple de Scikit-learn, et aussi un modèle SGDClassifier(loss='hinge'), qui correspond à un modèle SVM linéaire.

In [81]:
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.naive_bayes import ComplementNB
from sklearn.pipeline import Pipeline
from sklearn.model_selection import GridSearchCV
from sklearn.metrics import f1_score

import numpy as np
import pandas as pd
from sklearn.datasets import fetch_rcv1
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.pipeline import Pipeline
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.svm import SVC
from sklearn.metrics import classification_report
from sklearn.linear_model import SGDClassifier
from sklearn.feature_extraction.text import CountVectorizer, TfidfTransformer

pipeline1 = Pipeline(
    [
        ("vect", TfidfVectorizer()),
        ("clf", ComplementNB()),
    ]
)

pipeline2 = Pipeline(
    [
        ('tfidf', TfidfVectorizer()),
        ('clf', SGDClassifier(loss='hinge'))
    ]
)

pipeline3 = Pipeline(
    [
        ('vect', CountVectorizer()),
        ('tfidf', TfidfTransformer()),
        ('clf', SGDClassifier(loss='hinge'))
    ]
)

pipeline4 = Pipeline(
    [
        ('vect', CountVectorizer()),
        ('tfidf', TfidfTransformer()),
        ('clf', ComplementNB())
    ]
)

Pipeline(steps=[('vect', CountVectorizer()), ('tfidf', TfidfTransformer()),
                ('clf', ComplementNB())])

In [83]:
parameter_grid1 = {
    'vect__max_df': [0.2, 0.5],
    'vect__ngram_range': [(1,1), (1,2)],
    'vect__use_idf': [True, False],
    'vect__norm': ['l1', 'l2'],
    'clf__alpha': [10.0] 
}

parameter_grid2 = {
    'tfidf__max_df': [0.2, 0.5],
    'tfidf__use_idf': [True, False],
    'tfidf__ngram_range': [(1, 1), (1, 2)],
    'clf__penalty': ['l1', 'l2'],
    'clf__alpha': [0.01]
}

parameter_grid3 = {
    'vect__max_df': [0.2, 0.5],
    'vect__ngram_range': [(1, 1), (1, 2)],
    'tfidf__use_idf': [True, False],
    'tfidf__norm': ['l1', 'l2'],
    'clf__alpha': [0.0001, 0.001, 0.001],
    'clf__max_iter': [1000, 2000]
}

parameter_grid4 = {
    'vect__max_df': [0.2, 0.5],
    'vect__ngram_range': [(1, 1), (1, 2)],
    'tfidf__use_idf': [True, False],
    'tfidf__norm': ['l1', 'l2'],
    'clf__alpha': [0.001, 0.001]
}

In [93]:
from sklearn.model_selection import GridSearchCV
from sklearn.metrics import make_scorer, f1_score
import joblib

# Définir le score f1 pour la classe 'grain'
f1_scorer = make_scorer(f1_score, pos_label=1)

grid_search1 = GridSearchCV(pipeline1, parameter_grid1, scoring=f1_scorer, cv=5, n_jobs=-1)
grid_search1.fit(train_df['texte'], train_df['categorie'])

GridSearchCV(cv=5,
             estimator=Pipeline(steps=[('vect', TfidfVectorizer()),
                                       ('clf', ComplementNB())]),
             n_jobs=-1,
             param_grid={'clf__alpha': [10.0], 'vect__max_df': [0.2, 0.5],
                         'vect__ngram_range': [(1, 1), (1, 2)],
                         'vect__norm': ['l1', 'l2'],
                         'vect__use_idf': [True, False]},
             scoring=make_scorer(f1_score, response_method='predict', pos_label=1))

In [94]:
# Afficher les meilleurs hyperparamètres trouvés
print("Meilleurs hyperparamètres pour ComplementNB:")
print(grid_search1.best_params_)
print("Meilleur score F1 pour Pipeline 1:", grid_search1.best_score_)

Meilleurs hyperparamètres pour ComplementNB:
{'clf__alpha': 10.0, 'vect__max_df': 0.2, 'vect__ngram_range': (1, 1), 'vect__norm': 'l2', 'vect__use_idf': True}
Meilleur score F1 pour ComplementNB : 0.09782668731914076


In [95]:
grid_search2 = GridSearchCV(pipeline2, parameter_grid2, scoring=f1_scorer, cv=5, n_jobs=-1)
grid_search2.fit(train_df['texte'], train_df['categorie'])

GridSearchCV(cv=5,
             estimator=Pipeline(steps=[('tfidf', TfidfVectorizer()),
                                       ('clf', SGDClassifier())]),
             n_jobs=-1,
             param_grid={'clf__alpha': [0.01], 'clf__penalty': ['l1', 'l2'],
                         'tfidf__max_df': [0.2, 0.5],
                         'tfidf__ngram_range': [(1, 1), (1, 2)],
                         'tfidf__use_idf': [True, False]},
             scoring=make_scorer(f1_score, response_method='predict', pos_label=1))

In [96]:
# Afficher les meilleurs hyperparamètres trouvés
print("Meilleurs hyperparamètres pour SGD:")
print(grid_search2.best_params_)
print("Meilleur score F1 pour Pipeline 2 :", grid_search2.best_score_)

Meilleurs hyperparamètres pour SGD:
{'clf__alpha': 0.01, 'clf__penalty': 'l1', 'tfidf__max_df': 0.2, 'tfidf__ngram_range': (1, 1), 'tfidf__use_idf': True}
Meilleur score F1 pour ComplementNB : 0.0


In [97]:
grid_search3 = GridSearchCV(pipeline3, parameter_grid3, scoring=f1_scorer, cv=5, n_jobs=-1)
grid_search3.fit(train_df['texte'], train_df['categorie'])

GridSearchCV(cv=5,
             estimator=Pipeline(steps=[('vect', CountVectorizer()),
                                       ('tfidf', TfidfTransformer()),
                                       ('clf', SGDClassifier())]),
             n_jobs=-1,
             param_grid={'clf__alpha': [0.0001, 0.001, 0.001],
                         'clf__max_iter': [1000, 2000],
                         'tfidf__norm': ['l1', 'l2'],
                         'tfidf__use_idf': [True, False],
                         'vect__max_df': [0.2, 0.5],
                         'vect__ngram_range': [(1, 1), (1, 2)]},
             scoring=make_scorer(f1_score, response_method='predict', pos_label=1))

In [98]:
# Afficher les meilleurs hyperparamètres trouvés
print("Meilleurs hyperparamètres pour ComplementNB:")
print(grid_search3.best_params_)
print("Meilleur score F1 pour Pipeline 3 :", grid_search3.best_score_)

Meilleurs hyperparamètres pour ComplementNB:
{'clf__alpha': 0.0001, 'clf__max_iter': 2000, 'tfidf__norm': 'l2', 'tfidf__use_idf': True, 'vect__max_df': 0.5, 'vect__ngram_range': (1, 1)}
Meilleur score F1 pour ComplementNB : 0.9315986427410203


In [99]:
grid_search4 = GridSearchCV(pipeline4, parameter_grid4, scoring=f1_scorer, cv=5, n_jobs=-1)
grid_search4.fit(train_df['texte'], train_df['categorie'])

GridSearchCV(cv=5,
             estimator=Pipeline(steps=[('vect', CountVectorizer()),
                                       ('tfidf', TfidfTransformer()),
                                       ('clf', ComplementNB())]),
             n_jobs=-1,
             param_grid={'clf__alpha': [0.001, 0.001],
                         'tfidf__norm': ['l1', 'l2'],
                         'tfidf__use_idf': [True, False],
                         'vect__max_df': [0.2, 0.5],
                         'vect__ngram_range': [(1, 1), (1, 2)]},
             scoring=make_scorer(f1_score, response_method='predict', pos_label=1))

In [100]:
# Afficher les meilleurs hyperparamètres trouvés
print("Meilleurs hyperparamètres pour ComplementNB:")
print(grid_search4.best_params_)
print("Meilleur score F1 pour Pipeline 4 :", grid_search4.best_score_)

Meilleurs hyperparamètres pour ComplementNB:
{'clf__alpha': 0.001, 'tfidf__norm': 'l2', 'tfidf__use_idf': True, 'vect__max_df': 0.2, 'vect__ngram_range': (1, 2)}
Meilleur score F1 pour ComplementNB : 0.8458321213528283


Pipeline|F1 Score
---|---
1|0.0978
2|0.0
3|0.9316
4|0.8458

On peut voir que le pipeline 3 est le meilleur, avec un F1 Score de 0.9316. Les hyperparamètres optimaux sont les suivants: 
'clf__alpha': 0.0001,  
'clf__max_iter': 2000,  
'tfidf__norm': 'l2',  
'tfidf__use_idf': True,   
'vect__max_df': 0.5,  
'vect__ngram_range': (1, 1)

### 4. Quel est le score f1 de votre meilleure configuration trouvée ci-dessus, cette fois-ci sur les données de test ? Quels sont le rappel et la précision de la classe ‘grain’ ?

In [130]:
from sklearn.metrics import f1_score, classification_report

# Extraire les meilleurs hyperparamètres
result1_df = pd.DataFrame.from_dict(grid_search1.best_params_, orient='index', columns=['Valeur'])
result2_df = pd.DataFrame.from_dict(grid_search2.best_params_, orient='index', columns=['Valeur'])
result3_df = pd.DataFrame.from_dict(grid_search3.best_params_, orient='index', columns=['Valeur'])
result4_df = pd.DataFrame.from_dict(grid_search4.best_params_, orient='index', columns=['Valeur'])

# Prédictions sur les données de test
pred1 = grid_search1.predict(test_df['texte'])
pred2 = grid_search2.predict(test_df['texte'])
pred3 = grid_search3.predict(test_df['texte'])
pred4 = grid_search4.predict(test_df['texte'])

# Afficher les hyperparamètres optimaux
print(f"Hyper param pour 1 pipeline-> \n{result1_df}")
print(f"Hyper param pour 2 pipeline-> \n{result2_df}")
print(f"Hyper param pour 3 pipeline-> \n{result3_df}")
print(f"Hyper param pour 4 pipeline-> \n{result4_df}")

# Générer et afficher les rapports de classification
report1 = classification_report(test_df['categorie'], pred1, target_names=['non-grain', 'grain'], zero_division=0)
report2 = classification_report(test_df['categorie'], pred2, target_names=['non-grain', 'grain'], zero_division=0)
report3 = classification_report(test_df['categorie'], pred3, target_names=['non-grain', 'grain'], zero_division=0)
report4 = classification_report(test_df['categorie'], pred4, target_names=['non-grain', 'grain'], zero_division=0)


Hyper param pour 1 pipeline-> 
                   Valeur
clf__alpha           10.0
vect__max_df          0.2
vect__ngram_range  (1, 1)
vect__norm             l2
vect__use_idf        True
Hyper param pour 2 pipeline-> 
                    Valeur
clf__alpha            0.01
clf__penalty            l1
tfidf__max_df          0.2
tfidf__ngram_range  (1, 1)
tfidf__use_idf        True
Hyper param pour 3 pipeline-> 
                   Valeur
clf__alpha         0.0001
clf__max_iter        2000
tfidf__norm            l2
tfidf__use_idf       True
vect__max_df          0.5
vect__ngram_range  (1, 1)
Hyper param pour 4 pipeline-> 
                   Valeur
clf__alpha          0.001
tfidf__norm            l2
tfidf__use_idf       True
vect__max_df          0.2
vect__ngram_range  (1, 2)


In [132]:
# afficher les rapports 1 de perf
print(f"Classification Report for Pipeline 1:\n{report1}\n")

Classification Report for Pipeline 1:
              precision    recall  f1-score   support

   non-grain       0.95      1.00      0.97      1834
       grain       1.00      0.05      0.09       109

    accuracy                           0.95      1943
   macro avg       0.97      0.52      0.53      1943
weighted avg       0.95      0.95      0.92      1943




In [133]:
# afficher les rapports 2 de perf
print(f"Classification Report for Pipeline 2:\n{report2}\n")

Classification Report for Pipeline 2:
              precision    recall  f1-score   support

   non-grain       0.94      1.00      0.97      1834
       grain       0.00      0.00      0.00       109

    accuracy                           0.94      1943
   macro avg       0.47      0.50      0.49      1943
weighted avg       0.89      0.94      0.92      1943




In [134]:
# afficher les rapports 3 de perf
print(f"Classification Report for Pipeline 3:\n{report3}\n")

Classification Report for Pipeline 3:
              precision    recall  f1-score   support

   non-grain       0.99      1.00      0.99      1834
       grain       0.97      0.80      0.87       109

    accuracy                           0.99      1943
   macro avg       0.98      0.90      0.93      1943
weighted avg       0.99      0.99      0.99      1943




In [135]:
# afficher les rapports 4 de perf
print(f"Classification Report for Pipeline 4:\n{report4}\n")

Classification Report for Pipeline 4:
              precision    recall  f1-score   support

   non-grain       0.99      0.98      0.99      1834
       grain       0.72      0.86      0.78       109

    accuracy                           0.97      1943
   macro avg       0.85      0.92      0.88      1943
weighted avg       0.98      0.97      0.97      1943




In [139]:
from sklearn.metrics import f1_score, precision_score, recall_score, classification_report

# Calculer les métriques pour les meilleurs modèles
f1_1 = f1_score(test_df['categorie'], pred1, pos_label=1, average='macro')
precision_1 = precision_score(test_df['categorie'], pred1, pos_label=1, zero_division=0, average='macro')
recall_1 = recall_score(test_df['categorie'], pred1, pos_label=1, average='macro')

f1_2 = f1_score(test_df['categorie'], pred2, pos_label=1, average='macro')
precision_2 = precision_score(test_df['categorie'], pred2, pos_label=1, zero_division=0, average='macro')
recall_2 = recall_score(test_df['categorie'], pred2, pos_label=1, average='macro')

f1_3 = f1_score(test_df['categorie'], pred3, pos_label=1, average='macro')
precision_3 = precision_score(test_df['categorie'], pred3, pos_label=1, zero_division=0, average='macro')
recall_3 = recall_score(test_df['categorie'], pred3, pos_label=1, average='macro')

f1_4 = f1_score(test_df['categorie'], pred4, pos_label=1, average='macro')
precision_4 = precision_score(test_df['categorie'], pred4, pos_label=1, zero_division=0, average='macro')
recall_4 = recall_score(test_df['categorie'], pred4, pos_label=1, average='macro')

# Créer un DataFrame pour résumer les résultats
results = {
    'Pipeline': ['Pipeline 1', 'Pipeline 2', 'Pipeline 3', 'Pipeline 4'],
    'F1 Score': [f1_1, f1_2, f1_3, f1_4],
    'Precision': [precision_1, precision_2, precision_3, precision_4],
    'Recall': [recall_1, recall_2, recall_3, recall_4]
}

results_df = pd.DataFrame(results)

# Afficher le tableau des résultats
print("\nRésultats sur les données de test :")
print(results_df)


Résultats sur les données de test :
     Pipeline  F1 Score  Precision    Recall
0  Pipeline 1  0.530074   0.973168  0.522936
1  Pipeline 2  0.485571   0.471951  0.500000
2  Pipeline 3  0.933796   0.977397  0.898265
3  Pipeline 4  0.884536   0.854640  0.921105


>Quel est le score f1 de votre meilleure configuration trouvée ci-dessus, cette fois-ci sur les données de test ?  

Le meilleur score F1 sur les données de test est de 0.93 pour le modèle SGDClassifier avec la fonction de transformation de texte TfidfTransformer, qui transforme une matrice de termes en une représentation pondérée par TF-IDF (Term Frequency-Inverse Document Frequency). TF-IDF est une technique qui évalue l'importance d'un terme dans un document par rapport à un corpus, en pondérant la fréquence d'un terme par la fréquence inverse des documents. Les hyperparamètres optimaux pour ce pipeline sont les suivants :
Hyper param pour 3 pipeline->  
                   Valeur  
clf__alpha         0.0001  
clf__max_iter        2000  
tfidf__norm            l2  
tfidf__use_idf       True  
vect__max_df          0.5  
vect__ngram_range  (1, 1)  

Ces hyperparamètres indiquent que le modèle SGDClassifier a été régularisé avec un coefficient alpha de 0.0001 et a effectué jusqu'à 2000 itérations. Le TfidfTransformer a normalisé les vecteurs en utilisant la norme l2 et a utilisé les valeurs IDF (Inverse Document Frequency). Le CountVectorizer a été configuré pour inclure les termes présents dans au plus 50% des documents et pour considérer uniquement les unigrammes (n-grammes de longueur 1).



>Quels sont le rappel et la précision de la classe ‘grain’ ?

Class : Grain|Precision|Recall|F1-score
-|---------|------|-------
Pipeline 1|1|0.05|0.09
Pipeline 2|0|0|0
Pipeline 3|0.97|0.80|0.87
Pipeline 4|0.72|0.86|0.78

On peut voir que le pipeline 3 a le meilleur score F1 pour la classe 'grain' avec 0.87. Le rappel est de 0.80 et la précision est de 0.97. Cela signifie que le modèle du pipeline 3 est très efficace pour identifier les instances de la classe 'grain', avec une très haute précision et un bon rappel.

Le pipeline 4 a le meilleur rappel (Recall) pour la classe 'grain' avec 0.86. Cela signifie que ce modèle est très bon pour identifier les vrais positifs de la classe 'grain' (il détecte 86% des instances réelles de la classe 'grain'), mais il pourrait avoir un compromis en termes de précision, ce qui pourrait indiquer une plus grande proportion de faux positifs.

Le pipeline 1 a le plus bas score F1 pour la classe 'grain' avec 0.09. Le rappel est de 0.05 et la précision est de 1. Cela signifie que le modèle a prédit très peu de vrais positifs pour la classe 'grain', mais toutes les prédictions qu'il a faites pour cette classe étaient correctes. En d'autres termes, le modèle du pipeline 1 est extrêmement conservateur et prédit rarement la classe 'grain', mais quand il le fait, il est correct.

Le pipeline 2 a un score F1 de 0 pour la classe 'grain', ce qui signifie que le modèle n'a pas du tout prédit de vrais positifs pour cette classe. Cela indique une défaillance complète du modèle à identifier les instances de la classe 'grain'.

## Conclusion

Pour conclure, le pipeline 3 est le meilleur pour la classification des dépêches de Reuters-21578 en 'grain' et non-'grain'. Il a obtenu un score F1 de 0.87 sur les données de test, contre 0.93 sur les données d'entraînement, ce qui indique une très bonne performance générale avec une légère dégradation due à la généralisation sur des données non vues. Le rappel et la précision pour la classe 'grain' sont de 0.80 et 0.97 respectivement, ce qui montre que le modèle est très efficace pour identifier les instances de la classe 'grain' avec une très haute précision et un bon rappel.

SGDClassifier avec TfidfTransformer et CountVectorizer s'est révélé être la meilleure approche pour classer les dépêches de Reuters-21578 en 'grain' et non-'grain'. La combinaison de ces techniques permet d'obtenir une représentation vectorielle pondérée et normalisée des textes, facilitant ainsi une classification précise et efficace. Les hyperparamètres optimaux trouvés grâce à GridSearchCV ont contribué à maximiser la performance du modèle. Ce pipeline démontre une haute précision et un bon rappel, rendant le modèle fiable pour une application pratique.